# Sandbox Notebook

Experimental notebook for testing `home_media` package functionality.

Auto-reload is enabled to pick up changes to the package without restarting the kernel.

In [ ]:
# Enable auto-reload for the home_media package
%load_ext autoreload
%autoreload 2

In [ ]:
# Imports
import sys
from pathlib import Path
import yaml
import pandas as pd
from datetime import datetime

# Add parent directory to path so we can import home_media
sys.path.insert(0, str(Path.cwd().parent))

# Import home_media package
from home_media import (
    scan_directory,
    list_subdirectories,
    group_files_to_images,
    extract_base_name,
    extract_exif_metadata,
    ExifData,
    Image,
    ImageFile,
    FileRole,
    FileFormat,
)


## Load Configuration

In [ ]:
# Load config from config.yaml
config_path = Path.cwd().parent / "config.yaml"

with open(config_path, 'r') as f:
    config = yaml.safe_load(f)

photos_root = Path(config['photos_root_original'])

print("Configuration loaded:")
print(f"photos_root_original: {photos_root}")

## List Subdirectories in photos_root_original

In [ ]:
# Use home_media to list subdirectories
subdirs = list_subdirectories(photos_root)

print(f"Found {len(subdirs)} subdirectories in: {photos_root}\n")

for subdir in subdirs[:20]:  # Show first 20
    print(f"  📁 {subdir.name}")

if len(subdirs) > 20:
    print(f"\n  ... and {len(subdirs) - 20} more")

## Scan a Specific Subdirectory

Let's examine a specific subdirectory to see what images and files we have.

In [ ]:
# Choose a subdirectory to examine
subdir = "2025/01/01"
target_dir = photos_root / subdir

print(f"Scanning directory: {target_dir}")
print(f"Relative path: {subdir}")
print()

# Use home_media to scan and group files
images_df, files_df = scan_directory(target_dir, photos_root=photos_root, extract_exif=True)

print(f"Found {len(images_df)} images with {len(files_df)} total files")

## Images DataFrame

One row per image (moment in time), with aggregated information about all its files.

In [ ]:
# Display images DataFrame
print("Images DataFrame:")
print(f"Columns: {list(images_df.columns)}\n")

# Show first 10 images
images_df.head(10)

In [ ]:
# Summary statistics
print("Image Statistics:\n")
print(f"Total images: {len(images_df)}")
print(f"Images with RAW files: {images_df['has_raw'].sum()}")
print(f"Images with JPEG files: {images_df['has_jpeg'].sum()}")
print(f"Images with sidecars: {images_df['has_sidecar'].sum()}")
print(f"\nAverage files per image: {images_df['file_count'].mean():.1f}")
print(f"Max files per image: {images_df['file_count'].max()}")
print(f"\nTotal storage: {images_df['total_size_bytes'].sum() / (1024**3):.2f} GB")

In [ ]:
# Show outlier images with the most files
max_files = images_df['file_count'].max()
outliers = images_df[images_df['file_count'] == max_files]

print(f"Images with {max_files} files (outliers):\n")
for idx, row in outliers.iterrows():
    print(f"Base name: {row['base_name']}")
    print(f"  Subdirectory: {row['subdirectory']}")
    print(f"  File count: {row['file_count']}")
    print(f"  Suffixes: {row['suffixes']}")
    print(f"  Has RAW: {row['has_raw']}, Has JPEG: {row['has_jpeg']}, Has sidecar: {row['has_sidecar']}")
    print(f"  Total size: {row['total_size_bytes'] / (1024**2):.2f} MB")
    print()

# Also show top 5 images by file count
print("\nTop 5 images by file count:")
top_images = images_df.nlargest(5, 'file_count')[['base_name', 'subdirectory', 'file_count', 'suffixes', 'has_raw', 'has_jpeg', 'has_sidecar']]
top_images

## Files DataFrame

One row per file, linked to images by `base_name` and `subdirectory`.

In [ ]:
# Display files DataFrame
print("Files DataFrame:")
print(f"Columns: {list(files_df.columns)}\n")

# Show first 10 files
files_df.head(10)

In [ ]:
# File format breakdown
print("File Format Distribution:\n")
format_counts = files_df['format'].value_counts()
for format_type, count in format_counts.items():
    print(f"  {format_type}: {count}")

print("\nFile Role Distribution:\n")
role_counts = files_df['role'].value_counts()
for role, count in role_counts.items():
    print(f"  {role}: {count}")

## Working with Image Objects

For more complex operations, we can work with the `Image` objects directly.

In [ ]:
# Get file paths and group into Image objects
file_paths = [f for f in target_dir.iterdir() if f.is_file()]
images = group_files_to_images(file_paths, photos_root=photos_root)

print(f"Created {len(images)} Image objects\n")

# Examine the first image in detail
if images:
    img = images[0]
    print(f"Image: {img.base_name}")
    print(f"  Subdirectory: {img.subdirectory}")
    print(f"  File count: {img.file_count}")
    print(f"  Has RAW: {img.has_raw}")
    print(f"  Has JPEG: {img.has_jpeg}")
    print(f"  Has sidecar: {img.has_sidecar}")
    print(f"  Total size: {img.total_size_bytes:,} bytes")
    print(f"  Suffixes: {img.suffixes}")
    print(f"\n  Files:")
    for f in img.files:
        print(f"    - {f.filename}")
        print(f"      Role: {f.role.name}, Format: {f.format.value}, Size: {f.file_size_bytes:,} bytes")

## Filename Pattern Analysis

Let's see how the `extract_base_name()` function handles different filename patterns.

In [ ]:
# Test different filename patterns
test_filenames = [
    "2025-01-01_00-28-40.jpg",
    "2025-01-01_00-28-40.CR3",
    "2025-01-01_00-28-40_001.jpg",
    "2025-01-01_00-28-40.jpg.xmp",
    "PXL_20251210_200246684.RAW-01.COVER.jpg",
    "PXL_20251210_200246684.RAW-02.ORIGINAL.dng",
    "photo_edit.jpg",
]

print("Filename Pattern Analysis:\n")
for filename in test_filenames:
    base_name, suffix = extract_base_name(filename)
    print(f"{filename}")
    print(f"  base_name: {base_name}")
    print(f"  suffix: {suffix}")
    print()

## Images with Multiple Files

Find images that have multiple files (RAW+JPEG pairs, sidecars, etc.).

In [ ]:
# Filter images with multiple files
multi_file_images = images_df[images_df['file_count'] > 1]

print(f"Images with multiple files: {len(multi_file_images)} of {len(images_df)}\n")

# Show examples
print("Examples:")
for idx, row in multi_file_images.head(5).iterrows():
    print(f"\n{row['base_name']}:")
    print(f"  Files: {row['file_count']}")
    print(f"  Suffixes: {row['suffixes']}")
    print(f"  Has RAW: {row['has_raw']}, Has JPEG: {row['has_jpeg']}, Has sidecar: {row['has_sidecar']}")

## Canonical Naming Preview

Preview how files would be renamed using the canonical naming scheme: `YYYY/mm/dd/YYYY-mm-dd_HH-MM-SS` + suffix.

In the future, the capture time will be extracted from EXIF data.

In [ ]:
# Preview canonical naming for the first image
if images:
    img = images[0]

    # Simulate EXIF capture time (in real use, this comes from EXIF)
    # Try to parse from the base_name if it follows our standard format
    try:
        # Try to parse datetime from base_name like "2025-01-01_00-28-40"
        if len(img.base_name) >= 19 and img.base_name[10] == '_':
            simulated_capture_time = datetime.strptime(img.base_name[:19], "%Y-%m-%d_%H-%M-%S")
        else:
            simulated_capture_time = datetime(2025, 1, 1, 0, 28, 40)
    except Exception:
        simulated_capture_time = datetime(2025, 1, 1, 0, 28, 40)

    print("Current naming:")
    print(f"  Base name: {img.base_name}")
    print(f"  Subdirectory: {img.subdirectory}")
    print(f"  Files: {img.file_count}")

    print(f"\nCanonical naming (based on capture time: {simulated_capture_time}):")
    print(f"  Base name: {img.get_canonical_name(simulated_capture_time)}")
    print(f"  Subdirectory: {img.get_canonical_subdirectory(simulated_capture_time)}")

    print(f"\nFull canonical paths would be:")
    for f in img.files:
        canonical_path = (
            f"{img.get_canonical_subdirectory(simulated_capture_time)}/"
            f"{img.get_canonical_name(simulated_capture_time)}{f.suffix}"
        )
        print(f"  {canonical_path}")

## Query Examples

Use pandas to query the data in various ways.

In [ ]:
# Find images that only have RAW files (no JPEG)
raw_only = images_df[(images_df['has_raw']) & (~images_df['has_jpeg'])]
print(f"Images with only RAW (no JPEG): {len(raw_only)}")
raw_only.head()

In [ ]:
# Find images with XMP sidecars
with_sidecar = images_df[images_df['has_sidecar']]
print(f"Images with XMP sidecars: {len(with_sidecar)}")
with_sidecar.head()

In [ ]:
# Find all CR3 (Canon RAW) files
cr3_files = files_df[files_df['format'] == 'cr3']
print(f"Canon RAW (CR3) files: {len(cr3_files)}")
cr3_files.head()

In [ ]:
# Largest images by total size
largest_images = images_df.nlargest(10, 'total_size_bytes')[['base_name', 'file_count', 'total_size_bytes']]
largest_images['size_mb'] = largest_images['total_size_bytes'] / (1024**2)
print("Largest images by total size:\n")
largest_images

## EXIF Metadata Extraction

Test the new EXIF extraction functionality to populate Image metadata from the original files.

**Supported formats:**
- RAW files: CR2, CR3, NEF, ARW, DNG, RAF, ORF, RW2 (using exifread)
- Standard formats: JPEG, PNG, TIFF (using Pillow)

In [ ]:
# Test EXIF extraction on a single image
if images:
    test_image = images[0]

    print(f"Testing EXIF extraction on: {test_image.base_name}")
    print(f"Original file: {test_image.original_file.filename if test_image.original_file else 'None'}")
    print()

    # Before extraction
    print("Before EXIF extraction:")
    print(f"  captured_at: {test_image.captured_at}")
    print(f"  camera_make: {test_image.camera_make}")
    print(f"  camera_model: {test_image.camera_model}")
    print(f"  lens: {test_image.lens}")
    print(f"  GPS: {test_image.gps_latitude}, {test_image.gps_longitude}")
    print()

    # Extract EXIF
    success = test_image.populate_from_exif()

    # After extraction
    print(f"EXIF extraction {'successful' if success else 'failed'}")
    if success:
        print("\nAfter EXIF extraction:")
        print(f"  captured_at: {test_image.captured_at}")
        print(f"  camera_make: {test_image.camera_make}")
        print(f"  camera_model: {test_image.camera_model}")
        print(f"  lens: {test_image.lens}")
        print(f"  GPS: {test_image.gps_latitude}, {test_image.gps_longitude}")
        print(f"  title: {test_image.title}")
        print(f"  description: {test_image.description}")
        print(f"  rating: {test_image.rating}")

In [ ]:
# Test direct EXIF extraction from a file
if images and test_image.original_file:
    file_path = test_image.original_file.file_path
    
    print(f"Direct EXIF extraction from: {file_path.name}")
    print()
    
    exif_data = extract_exif_metadata(file_path)
    
    if exif_data:
        print("Extracted EXIF data:")
        print(f"  captured_at: {exif_data.captured_at}")
        print(f"  camera_make: {exif_data.camera_make}")
        print(f"  camera_model: {exif_data.camera_model}")
        print(f"  lens: {exif_data.lens}")
        print(f"  GPS coordinates: ({exif_data.gps_latitude}, {exif_data.gps_longitude})")
        print(f"  title: {exif_data.title}")
        print(f"  description: {exif_data.description}")
        print(f"  rating: {exif_data.rating}")
    else:
        print("No EXIF data found or extraction failed")

### Scan Directory with EXIF Extraction

Test the integrated EXIF extraction in `scan_directory()` - this will populate metadata for all images automatically.

In [ ]:
# Scan directory with EXIF extraction enabled
print("Scanning with EXIF extraction enabled...")
print("Note: This may be slower for large directories.\n")

images_with_exif_df, files_with_exif_df = scan_directory(
    target_dir, 
    photos_root=photos_root,
    extract_exif=True
)

print(f"Scanned {len(images_with_exif_df)} images")
print()

# Show images with populated EXIF data
print("Sample of images with EXIF data:")
exif_columns = ['base_name', 'captured_at', 'camera_make', 'camera_model']
images_with_exif_df[exif_columns].head(10)

In [ ]:
# EXIF data coverage statistics
print("EXIF Data Coverage:\n")

total_images = len(images_with_exif_df)
with_capture_time = images_with_exif_df['captured_at'].notna().sum()
with_camera_make = images_with_exif_df['camera_make'].notna().sum()
with_camera_model = images_with_exif_df['camera_model'].notna().sum()

print(f"Total images: {total_images}")
print(f"With capture time: {with_capture_time} ({with_capture_time/total_images*100:.1f}%)")
print(f"With camera make: {with_camera_make} ({with_camera_make/total_images*100:.1f}%)")
print(f"With camera model: {with_camera_model} ({with_camera_model/total_images*100:.1f}%)")

# Show unique cameras found
if with_camera_make > 0:
    print("\nUnique cameras found:")
    camera_combos = images_with_exif_df[['camera_make', 'camera_model']].dropna()
    unique_cameras = camera_combos.drop_duplicates()
    for _, row in unique_cameras.iterrows():
        print(f"  {row['camera_make']} {row['camera_model']}")

In [ ]:
# Compare canonical naming with EXIF data vs filename-based
if with_capture_time > 0:
    # Get first image with EXIF capture time
    sample = images_with_exif_df[images_with_exif_df['captured_at'].notna()].iloc[0]

    print("Canonical naming comparison:\n")
    print(f"Original base_name: {sample['base_name']}")
    print(f"Original subdirectory: {sample['subdirectory']}")
    print()
    print(f"EXIF captured_at: {sample['captured_at']}")

    # Create an Image object to use the canonical naming methods
    temp_img = Image(
        base_name=sample['base_name'],
        subdirectory=sample['subdirectory'],
        captured_at=sample['captured_at']
    )

    print(f"\nCanonical name (from EXIF): {temp_img.get_canonical_name()}")
    print(f"Canonical subdirectory (from EXIF): {temp_img.get_canonical_subdirectory()}")

    # Show how files would be renamed
    print("\nFiles for this image would be renamed to:")
    sample_files = files_with_exif_df[
        (files_with_exif_df['base_name'] == sample['base_name']) &
        (files_with_exif_df['subdirectory'] == sample['subdirectory'])
    ]

    for _, file_row in sample_files.iterrows():
        # Extract suffix from filename
        suffix = file_row['filename'][len(sample['base_name']):]
        canonical_path = f"{temp_img.get_canonical_subdirectory()}/{temp_img.get_canonical_name()}{suffix}"
        print(f"  {canonical_path}")

### Scan with Full Metadata Extraction

Test `scan_directory()` with all metadata options enabled: EXIF, file hashes, and dimensions.

In [ ]:
# Scan with full metadata extraction
# Note: This will be slower due to hash calculation and dimension extraction
print("Scanning with full metadata extraction...")
print("This includes: EXIF, file hashes (SHA256), and image dimensions")
print("Note: This may take a while for large directories.\n")

images_full_df, files_full_df = scan_directory(
    target_dir,
    photos_root=photos_root,
    extract_exif=True,
    calculate_hash=True,
    extract_dimensions=True
)

print(f"Scanned {len(images_full_df)} images with {len(files_full_df)} files")
print()

# Show sample with all metadata
print("Sample of files with full metadata:")
metadata_columns = ['filename', 'format', 'role', 'width', 'height', 'file_size_bytes', 'file_hash']
files_full_df[metadata_columns].head(10)

In [ ]:
# Find potential duplicate files using hash
duplicates = files_full_df[files_full_df['file_hash'].notna()].groupby('file_hash').filter(lambda x: len(x) > 1)

if len(duplicates) > 0:
    print(f"Found {len(duplicates)} files with duplicate hashes:\n")
    
    # Group by hash and show duplicates
    for hash_value, group in duplicates.groupby('file_hash'):
        print(f"Hash: {hash_value}")
        for _, file_row in group.iterrows():
            print(f"  - {file_row['filename']} ({file_row['file_size_bytes']:,} bytes)")
            print(f"    Path: {file_row['file_path']}")
        print()
else:
    print("No duplicate files found (based on hash)")

In [ ]:
# Metadata coverage statistics
print("Metadata Coverage:\n")

total_files = len(files_full_df)
with_hash = files_full_df['file_hash'].notna().sum()
with_dimensions = files_full_df['width'].notna().sum()

print(f"Total files: {total_files}")
print(f"With hash: {with_hash} ({with_hash/total_files*100:.1f}%)")
print(f"With dimensions: {with_dimensions} ({with_dimensions/total_files*100:.1f}%)")

# Show dimension statistics for image files only
image_files = files_full_df[files_full_df['width'].notna()]
if len(image_files) > 0:
    print("\nImage Dimension Statistics:")
    print(f"  Width range: {image_files['width'].min()} - {image_files['width'].max()} pixels")
    print(f"  Height range: {image_files['height'].min()} - {image_files['height'].max()} pixels")
    print(f"  Average dimensions: {image_files['width'].mean():.0f}x{image_files['height'].mean():.0f} pixels")
    
    # Calculate megapixels
    image_files_copy = image_files.copy()
    image_files_copy['megapixels'] = (image_files_copy['width'] * image_files_copy['height']) / 1_000_000
    print(f"  Average megapixels: {image_files_copy['megapixels'].mean():.2f} MP")
    print(f"  Max megapixels: {image_files_copy['megapixels'].max():.2f} MP")
    
    # Show unique resolutions
    unique_resolutions = image_files[['width', 'height']].drop_duplicates().sort_values(['width', 'height'], ascending=False)
    print(f"\nUnique resolutions found: {len(unique_resolutions)}")
    for _, row in unique_resolutions.head(10).iterrows():
        mp = (row['width'] * row['height']) / 1_000_000
        print(f"  {int(row['width'])}x{int(row['height'])} ({mp:.2f} MP)")

In [ ]:
# Test hash and dimensions extraction on a single file
if images:
    test_image = images[0]
    test_file = test_image.files[0] if test_image.files else None
    
    if test_file:
        print(f"Testing on file: {test_file.filename}")
        print(f"Format: {test_file.format.value}, Role: {test_file.role.name}")
        print()
        
        # Before extraction
        print("Before extraction:")
        print(f"  file_hash: {test_file.file_hash}")
        print(f"  dimensions: {test_file.width}x{test_file.height}")
        print()
        
        # Extract hash
        print("Calculating SHA256 hash...")
        hash_success = test_file.populate_hash()
        print(f"  Success: {hash_success}")
        if hash_success:
            print(f"  Hash: {test_file.file_hash}")
        print()
        
        # Extract dimensions
        print("Extracting dimensions...")
        dim_success = test_file.populate_dimensions()
        print(f"  Success: {dim_success}")
        if dim_success:
            print(f"  Dimensions: {test_file.width}x{test_file.height} pixels")
            aspect_ratio = test_file.width / test_file.height if test_file.height else 0
            print(f"  Aspect ratio: {aspect_ratio:.3f}")
            megapixels = (test_file.width * test_file.height) / 1_000_000 if test_file.width and test_file.height else 0
            print(f"  Megapixels: {megapixels:.2f} MP")

## File Hash and Dimensions

Test the new file hash and dimensions extraction functionality.

In [ ]:
# Metadata coverage statistics
print("Metadata Coverage:\n")

total_files = len(files_full_df)
with_hash = files_full_df['file_hash'].notna().sum()
with_dimensions = files_full_df['width'].notna().sum()

print(f"Total files: {total_files}")
print(f"With hash: {with_hash} ({with_hash/total_files*100:.1f}%)")
print(f"With dimensions: {with_dimensions} ({with_dimensions/total_files*100:.1f}%)")

# Show dimension statistics for image files only
image_files = files_full_df[files_full_df['width'].notna()]
if len(image_files) > 0:
    print("\nImage Dimension Statistics:")
    print(f"  Width range: {image_files['width'].min()} - {image_files['width'].max()} pixels")
    print(f"  Height range: {image_files['height'].min()} - {image_files['height'].max()} pixels")
    print(f"  Average dimensions: {image_files['width'].mean():.0f}x{image_files['height'].mean():.0f} pixels")
    
    # Calculate megapixels
    image_files_copy = image_files.copy()
    image_files_copy['megapixels'] = (image_files_copy['width'] * image_files_copy['height']) / 1_000_000
    print(f"  Average megapixels: {image_files_copy['megapixels'].mean():.2f} MP")
    print(f"  Max megapixels: {image_files_copy['megapixels'].max():.2f} MP")
    
    # Show unique resolutions
    unique_resolutions = image_files[['width', 'height']].drop_duplicates().sort_values(['width', 'height'], ascending=False)
    print(f"\nUnique resolutions found: {len(unique_resolutions)}")
    for _, row in unique_resolutions.head(10).iterrows():
        mp = (row['width'] * row['height']) / 1_000_000
        print(f"  {int(row['width'])}x{int(row['height'])} ({mp:.2f} MP)")